# MLflow Training Tutorial (la copie)

In [1]:
def train(labels_array, nb_epochs, nb_patience):

    import tensorflow as tf
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Conv2D, GlobalAveragePooling2D, GlobalMaxPooling2D, MaxPooling2D, AveragePooling2D, Activation, Dropout, Flatten, Dense
    from tensorflow.keras.callbacks import EarlyStopping
    from tensorflow.keras.preprocessing import image

    import mlflow
    import mlflow.tensorflow

    # Telechargement du ZIP
    from modules.loader import Loader

    loader = Loader(
        "https://stdatalake010.blob.core.windows.net/public/cifar-100.zip",
        '../datas/ZIP/',
        extraction_target='../datas/RAW/'
    )
    loader.ensure_data_loaded()

    # Extraction du jeu de donnees
    from modules.splitting import Splitting

    labels_array = ['apple', 'bee']

    TRAIN_DATA_DIR = Splitting.copie_dossiers(
        '../datas/RAW/train',
        labels_array,
        500,
        explorer=False
    )

    print(TRAIN_DATA_DIR)

    # Chargement des images
    image_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(validation_split=0.2)
    TRAIN_IMAGE_SIZE = 32
    TRAIN_BATCH_SIZE = 64

    train_generator = image_data_generator.flow_from_directory(
        TRAIN_DATA_DIR,
        target_size=(TRAIN_IMAGE_SIZE, TRAIN_IMAGE_SIZE),
        batch_size=TRAIN_BATCH_SIZE,
        class_mode='categorical',
        subset='training')
    
    validation_generator = image_data_generator.flow_from_directory(
        TRAIN_DATA_DIR, # same directory as training data
        target_size=(TRAIN_IMAGE_SIZE, TRAIN_IMAGE_SIZE),
        batch_size=TRAIN_BATCH_SIZE,
        class_mode='categorical',
        subset='validation')

    with mlflow.start_run():

        model = Sequential()
        model.add(Conv2D(32, kernel_size=3, activation='elu', kernel_initializer='he_uniform', padding='same', input_shape=(32,32,3)))
        #Toujours à la fin
        model.add(Flatten())
        model.add(Dense(2, activation='softmax'))

        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

        es_callback = EarlyStopping(monitor='val_loss', patience=nb_patience)
        training = model.fit(train_generator, epochs=nb_epochs, callbacks=[es_callback], validation_data=validation_generator, shuffle=False)
        mlflow.tensorflow.autolog()

In [2]:
labels_array = ['apple', 'bee']
nb_epochs = 15
nb_patience = 4

train(labels_array, nb_epochs, nb_patience)

Le fichier ZIP existe déjà
Le dossier d:\simplon\datas\RAW\train_apple_bee existe déjà !
d:\simplon\datas\RAW\train_apple_bee
Found 800 images belonging to 2 classes.
Found 200 images belonging to 2 classes.
Epoch 1/15
13/13 [==============================] - 1s 56ms/step - loss: 654.0524 - accuracy: 0.4837 - val_loss: 121.9647 - val_accuracy: 0.7300
Epoch 2/15
13/13 [==============================] - 0s 38ms/step - loss: 113.0606 - accuracy: 0.7450 - val_loss: 97.6988 - val_accuracy: 0.8050
Epoch 3/15
13/13 [==============================] - 0s 34ms/step - loss: 40.0981 - accuracy: 0.8725 - val_loss: 50.1241 - val_accuracy: 0.8250
Epoch 4/15
13/13 [==============================] - 0s 34ms/step - loss: 30.1505 - accuracy: 0.8775 - val_loss: 53.6862 - val_accuracy: 0.8350
Epoch 5/15
13/13 [==============================] - 0s 35ms/step - loss: 19.2246 - accuracy: 0.9175 - val_loss: 32.8592 - val_accuracy: 0.8400
Epoch 6/15
13/13 [==============================] - 0s 35ms/step - loss: 9